Библиотека selenium используется для автоматизации взаимодействия с веб-браузером

In [ ]:
!pip install selenium

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import shapely
from shapely.ops import snap, unary_union
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import pandas as pd
import json
import plotly.express as px
import ast
import zipfile
from collections import Counter
from bs4 import BeautifulSoup as bs
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd
import re
from collections import defaultdict

In [ ]:
!pip install pandas geopandas plotly

скачивает файл с границами регионов России в формате GeoJSON и сохраняет его

In [ ]:
!wget https://raw.githubusercontent.com/hairymax/offline_russia_plotly/main/data/russia_regions.geojson -O russia_regions.geojson

--2025-06-01 20:24:51--  https://raw.githubusercontent.com/hairymax/offline_russia_plotly/main/data/russia_regions.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26692099 (25M) [text/plain]
Saving to: ‘russia_regions.geojson’

russia_regions.geoj 100%[===================>]  25.46M   117MB/s    in 0.2s    

2025-06-01 20:24:51 (117 MB/s) - ‘russia_regions.geojson’ saved [26692099/26692099]



In [ ]:
#загружаем GeoJSON
with open('russia_regions.geojson', encoding='utf-8') as f:
    geojson = json.load(f)

In [ ]:
with zipfile.ZipFile('hh.zip', 'r') as z:
    with z.open('hh.csv') as f:
        df2 = pd.read_csv('/content/hh.zip', encoding='utf-8-sig')
df2.head(3)

,data_source,v_id,id,date_published,is_open,name,description,salary_from,salary_to,salary_currency,...,region_id,region_name,region_district_id,region_district_name,role_id,role_name,skills,hard_skills,soft_skills,experience_category
0,superjob,002c181d-02ff-bc9c-0a61-84f51501f72a,ddc0e7a2-3fb9-11ed-a9b7-dbfed3997e57,2023-10-09,1,Инженер по пусконаладке станков / Сервисный ин...,Наладка и проведение испытаний балансировочных...,80000,100000,RUB,...,347,Москва,1,Центральный,0,Инженер по пусконаладке станков / Сервисный ин...,NaN,NaN,NaN,1-3 года
1,superjob,00efeefa-3424-b47d-d29f-ca7baca22653,95e44fd0-3d06-11ee-b683-dbfed3997e57,2023-09-09,1,Инженер (2 категории),Обязанности:- Монтаж/демонтаж систем широкопол...,60000,120000,RUB,...,578,Ханты-Мансийский АО - Югра,5,Уральский,0,Инженер (2 категории),NaN,NaN,NaN,1-3 года
2,superjob,01202c78-e3e4-4d4a-f42d-001fdd73ed51,17600752-d468-11ed-b282-dbfed3997e57,2023-08-31,0,Начальник кабинета автоматизированных систем у...,Условия: Поликлиника находится в историческом...,100000,100000,RUB,...,475,Санкт-Петербург,2,Северо-Западный,0,Начальник кабинета автоматизированных систем у...,NaN,NaN,NaN,1-3 года


In [ ]:
#уникальные регионы из датафрейма
hz = sorted(df2['region_name'].dropna().unique())
print("Регионы из датафрейма:")
for region in hz:
    print(region)
    break

Регионы из датафрейма:
Аккурган


----------

In [ ]:
#функция нормализации
def normalize_region(name):
    return name.lower()

#получаем уникальные регионы из df2 и приводим к нижнему регистру
df_regions = df2['region_name'].dropna().unique()
normalized_df_regions = {normalize_region(r): r for r in df_regions}

In [ ]:
#получаем список регионов
geojson_regions = [feature['properties']['region'] for feature in geojson['features']]

#cоздаём map для привязки
geojson_map = {r.lower(): r for r in geojson_regions}

#печатаем регионы
print("\nРегионы из GeoJSON (с маленькой буквы):")
for region in geojson_regions:
    print(region.lower())
    break

normalized_geojson_regions = {normalize_region(r): r for r in geojson_regions}


Регионы из GeoJSON (с маленькой буквы):
пензенская область


In [ ]:
#ищем несовпадения
missing_in_geojson = [original for norm, original in normalized_df_regions.items() if norm not in normalized_geojson_regions]
missing_in_df = [original for norm, original in normalized_geojson_regions.items() if norm not in normalized_df_regions]

print("\nРегионы из DataFrame, которых нет в GeoJSON:")
for region in missing_in_geojson:
    print(region)
    break


Регионы из DataFrame, которых нет в GeoJSON:
Ханты-Мансийский АО - Югра


In [ ]:
#объединяем навыки
def parse_skills(row):
    all_skills = []
    for col in ['skills', 'hard_skills', 'soft_skills']:
        value = row.get(col, '')
        if pd.isnull(value):
            continue
        if isinstance(value, str) and value.startswith('['):
            try:
                parsed = ast.literal_eval(value)
            except:
                parsed = value.split(',')
        elif isinstance(value, str):
            parsed = value.split(',')
        else:
            parsed = value
        all_skills.extend([s.strip() for s in parsed if s.strip()])
    return all_skills

-----

In [ ]:
df2['all_skills'] = df2.apply(parse_skills, axis=1)

In [ ]:
#очищаем названия регионов (заменяем крайние пробелы и г.Москва  к приемру на Москва)
df2['region_clean'] = df2['region_name'].astype(str).str.replace('г. ', '', regex=False).str.lower().str.strip()

Проходится по спискам навыков (skills_lists) внутри каждой группы региона
Убирает None и не-списки
Склеивает всё в один список, считает частоты, берёт топ-10
Возвращает строку типа: "Python (124), Excel (99)"

In [ ]:
#считаем топ-10 навыков для каждого региона
def top_skills_for_region(skills_lists):
    all_skills = [skill for sublist in skills_lists if isinstance(sublist, list) for skill in sublist]
    top = Counter(all_skills).most_common(10)
    return ', '.join([f"{skill} ({count})" for skill, count in top])

Группировка по региону

In [ ]:
region_grouped = df2.groupby('region_clean')['all_skills'].apply(top_skills_for_region).reset_index()
region_grouped.columns = ['region_name', 'top_skills']

Подсчёт количества вакансий

In [ ]:
#добавим количество вакансий по региону
vacancy_counts = df2.groupby('region_clean').size().reset_index(name='vacancy_count')
vacancy_counts.columns = ['region_name', 'vacancy_count']

Объединение

In [ ]:
#объединяем всё в один датафрейм
region_df = pd.merge(vacancy_counts, region_grouped, on='region_name')
region_df

,region_name,vacancy_count,top_skills
0,nan,702,"ответственность проактивность (56), целеустрем..."
1,аккурган,1,"поиск привлечение новых клиентов (2), базовая ..."
2,аксай (казахстан),5,"коммуникативная грамотность (4), ответственнос..."
3,аксу (павлодар.обл),3,
4,актау,85,"коммуникативная грамотность (56), базовая инст..."
...,...,...,...
217,шымкент,239,"коммуникативная грамотность (144), работа кома..."
218,экибастуз,8,"выполнение ремонтных работ (2), техническая по..."
219,ямало-ненецкий ао,434,"работа команде (186), базовая инструментальная..."
220,янгиюль,1,


формируется датафрейм region_df с колонками:

region_name

vacancy_count

top_skills (в формате "Навык (кол-во), Навык2 (кол-во2), ...")

-----

ПРИВЯЗКА РЕГИОНОВ К GeoJSON
Это нужно для choropleth, чтобы region_name в locations= совпадал с featureidkey='properties.region'

In [ ]:
geojson_regions = [feature['properties']['region'] for feature in geojson['features']]
normalized_geojson_map = {normalize_region(r): r for r in geojson_regions}

region_df['region_display_name'] = region_df['region_name'].apply(normalize_region).map(normalized_geojson_map)

In [ ]:
#aункция для извлечения навыков и их количества из строки формата "навык (число), навык (число)"
def extract_skill_counts(skill_str):
    if not isinstance(skill_str, str):
        return []
    pattern = re.compile(r"(.+?)\s*\((\d+)\)")
    skills_counts = []
    for part in skill_str.split(','):
        part = part.strip()
        match = pattern.match(part)
        if match:
            skill = match.group(1).strip()
            count = int(match.group(2))
            skills_counts.append((skill, count))
        else:
            # Если нет числа — считаем количество 1
            skills_counts.append((part, 1))
    return sorted(skills_counts, key=lambda x: x[1], reverse=True)

In [ ]:
#подсчёт всех навыков и их упоминаний в списках навыков для каждого региона
def count_all_skills_for_region(skills_lists):
    all_skills = [skill for sublist in skills_lists if isinstance(sublist, list) for skill in sublist]
    return Counter(all_skills)

In [ ]:
#считаем полный счётчик навыков по регионам
region_skill_counts = df2.groupby('region_clean')['all_skills'].apply(count_all_skills_for_region).reset_index()
region_skill_counts = region_skill_counts.rename(columns={'region_clean': 'region_name', 'all_skills': 'all_skills_counter'})

In [ ]:
#добавляем колонки с общим числом уникальных навыков и общим числом упоминаний
region_skill_counts['total_skills_in_region'] = region_skill_counts['all_skills_counter'].apply(lambda x: len(x) if isinstance(x, dict) else 0)
region_skill_counts['total_skill_mentions'] = region_skill_counts['all_skills_counter'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else 0)

In [ ]:
region_df = region_df.merge(
    region_skill_counts[['region_name', 'total_skills_in_region', 'total_skill_mentions']],
    on='region_name',
    how='left'
)

In [ ]:
region_df

,region_name,vacancy_count,top_skills,region_display_name,total_skills_in_region,total_skill_mentions
0,nan,702,"ответственность проактивность (56), целеустрем...",NaN,0,0
1,nan,702,"ответственность проактивность (56), целеустрем...",NaN,0,0
2,nan,702,"ответственность проактивность (56), целеустрем...",NaN,0,0
3,nan,702,"ответственность проактивность (56), целеустрем...",NaN,0,0
4,nan,702,"ответственность проактивность (56), целеустрем...",NaN,0,0
...,...,...,...,...,...,...
183811,ярославская область,1079,"работа команде (368), коммуникативная грамотно...",Ярославская область,0,0
183812,ярославская область,1079,"работа команде (368), коммуникативная грамотно...",Ярославская область,0,0
183813,ярославская область,1079,"работа команде (368), коммуникативная грамотно...",Ярославская область,0,0
183814,ярославская область,1079,"работа команде (368), коммуникативная грамотно...",Ярославская область,0,0


In [ ]:
#преобразуем колонку top_skills в список (skill, count)
region_df['top_skills_counts'] = region_df['top_skills'].apply(extract_skill_counts)

In [ ]:
#функция создания текста для подсказки на карте
def make_hover_text(row):
    skills_data = row['top_skills_counts']
    vacancy_count = row['vacancy_count']
    region_name = row.get('region_display_name', row.get('region_name', ''))

    if not skills_data or vacancy_count == 0:
        skills_text = "<i>Нет данных по навыкам</i>"
    else:
        skills_text = "<br>".join([
            f" <b>{skill}</b> — {count} ({count / vacancy_count:.1%})"
            for skill, count in skills_data
        ])

    return (
        f"<b>Регион:</b> {region_name}<br>"
        f" <b>Всего вакансий в регионе:</b> {vacancy_count}<br>"
        f"<b>Топ навыки:</b><br>{skills_text}"
    )

region_df['hover_text'] = region_df.apply(make_hover_text, axis=1)

In [ ]:
#построение карты с кастомным hover (используем колонку hover_text для всплывающей подсказки)
fig = px.choropleth(
    region_df,
    geojson=geojson,
    locations='region_display_name',
    featureidkey='properties.region',
    color='vacancy_count',
    hover_name=None,  # отключаем стандартное имя
    hover_data={'hover_text': True, 'region_display_name': False, 'vacancy_count': False, 'top_skills': False},
    title='Карта вакансий и топ-навыков по регионам России',
    color_continuous_scale='Blues',
)

fig.update_geos(fitbounds="locations", visible=False)


fig.update_traces(
    hovertemplate='%{customdata[0]}<extra></extra>'
)

fig.update_layout(
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    hoverlabel=dict(
        bgcolor='rgba(255,255,255,0.95)',  # почти прозрачный
        font_size=13,
        font_family="Arial"
    )
)

fig.show()

Buffered data was truncated after reaching the output size limit.

In [ ]:
#формируем таблицу с ключевой инфой
region_summary = region_df[['region_display_name', 'vacancy_count', 'total_skills_in_region', 'top_skills']].copy()
region_summary = region_summary.rename(columns={
    'region_display_name': 'Регион',
    'vacancy_count': 'Всего акансий в регионе',
    'total_skills_in_region': 'Уникальных навыков',
    'top_skills': 'Топ навыки'
})

region_summary.head(10)

,Регион,Всего акансий в регионе,Уникальных навыков,Топ навыки
0,NaN,702,0,"ответственность проактивность (56), целеустрем..."
1,NaN,702,0,"ответственность проактивность (56), целеустрем..."
2,NaN,702,0,"ответственность проактивность (56), целеустрем..."
3,NaN,702,0,"ответственность проактивность (56), целеустрем..."
4,NaN,702,0,"ответственность проактивность (56), целеустрем..."
5,NaN,702,0,"ответственность проактивность (56), целеустрем..."
6,NaN,702,0,"ответственность проактивность (56), целеустрем..."
7,NaN,702,0,"ответственность проактивность (56), целеустрем..."
8,NaN,702,0,"ответственность проактивность (56), целеустрем..."
9,NaN,702,0,"ответственность проактивность (56), целеустрем..."
